In [1]:
import pandas as pd
import numpy as np
from time import time
from tqdm import tqdm
tqdm.pandas()

model_name = 'seresnext101'
fold = 2
checkpoint = '17600'
algo = 'dist_global_org'

nums = [model_name,fold,
        checkpoint,
        model_name,
        fold,algo]

In [3]:
top20 = pd.read_csv('../WC_result/{0}_{1}/out_{2}/{3}_sub_fold{4}_{5}.csv'.format(*nums), header=None)
top20 = top20.set_index(0)

In [4]:
enc = pd.read_csv('../WC_result/{0}_{1}/out_{2}/encoding_org_img.csv'.format(*nums), header=None)

enc = enc.set_index(0)

enc['embeddings'] = enc.values.tolist()

enc = enc.reset_index()

enc = enc.iloc[:, [0, 2050-1]]

enc.columns = ['img', 'embeddings']

enc.head(1)

Metric can be "angular", "euclidean", "manhattan", "hamming", or "dot"

In [11]:
from annoy import AnnoyIndex

f = len(enc['embeddings'][0])
t = AnnoyIndex(f, metric='angular')
ntree = 1000

for i, vector in tqdm(enumerate(enc['embeddings'])):
    t.add_item(i, vector)
_  = t.build(ntree)

5347it [00:00, 15882.44it/s]


In [12]:
def get_similar_images_annoy(img_index):
    t0 = time()
    v, f  = enc.iloc[img_index, [0, 1]]
    similar_img_ids = t.get_nns_by_item(img_index, 21)
    return v, f, enc.iloc[similar_img_ids].iloc[1:]

In [13]:
sample_idx = 18  # 166, # 302

v, f, s = get_similar_images_annoy(sample_idx)

In [14]:
test = enc[enc.img.isin(top20.index)]

In [15]:
test = test.reset_index() 

In [16]:
test.columns = ['id', 'img' , 'embeddings']

In [17]:
def get_top20_imgs(x):
    get_similar_images_annoy(x['id'])
    _, _, s = get_similar_images_annoy(x['id'])
    return s['img'].tolist()

In [18]:
test['top20imgs'] = test[test.img.isin(top20.index)].apply(lambda x: get_top20_imgs(x), axis=1)

In [19]:
test.head()

,id,img,embeddings,top20imgs
0,0,PM-WWA-20180811-093.jpg,"[0.10390169, -1.9826837e-05, -0.031419944, -0....","[PM-WWA-20160408-598.jpg, PM-WWA-20100723-339...."
1,18,PM-WWA-20170710-031.jpg,"[-0.007411923000000001, 2.2409202e-05, 0.01138...","[PM-WWA-20160319-207.jpg, PM-WWA-20110724-031...."
2,19,PM-WWA-20170622-226.jpg,"[-0.009813612, -6.898617400000001e-06, 0.06699...","[PM-WWA-20170625-283.jpg, PM-WWA-20060819-009...."
3,20,PM-WWA-20180813-271.jpg,"[-0.00257763, 6.6202214e-05, -0.012052906, -0....","[PM-WWA-20060818-192.jpg, PM-WWA-20080516-102...."
4,21,PM-WWA-20180506-348.jpg,"[0.15255915, -2.6346093e-05, 0.06690723, -0.01...","[PM-WWA-20060618-010.jpg, PM-WWA-20110720-104...."


In [20]:
pd.DataFrame(test.img).join(test['top20imgs'].apply(pd.Series)).to_csv('annoy_angular.csv', header=False, index=False)